In [83]:
# import libraries
import seaborn as sbs
import pandas as pd

# import data using code given
from Bio import SeqIO
human_genome = SeqIO.parse("GCA_000001405.28_GRCh38.p13_genomic.fna", "fasta")

In [57]:
# used code given to help loop through the entire chromosome across
# all 15-mers and find the total number of subsequences with more than 2 N's
less_than_two_n = 0
for chromosome in human_genome:
    if chromosome.name == "CM000664.2":
        sequence = str(chromosome.seq).lower().encode('utf8', errors='ignore')
        s_len = len(sequence)

        # if length of sequence <= 15, then do not enter while loop
        if s_len <= 15 and sequence.count('n'.encode('utf8')) < 3:
            less_than_two_n += 1
            continue
        # if length of sequence > 15, evaluate each 15-char substring
        i = 0
        while i + 15 <= s_len:
            subseq = sequence[i:i+15]
            if subseq.count('n'.encode('utf8')) < 3: less_than_two_n += 1
            i += 1
print(less_than_two_n)

240548031


In [58]:
# estimate the number of distinct 15-mers in the reference genome's chromosome 2
# using method discussed in class
# using 100 hash functions from the given family in a single pass through the sequences
p = 2_549_536_629_329
bits_48 = 2 ** 48 - 1
scale = 0x07ffffffff
from hashlib import sha256
def get_ath_hash(a):
    def my_hash(subseq):
        return (((int(sha256(subseq.encode("utf8")).hexdigest(), 16) % bits_48) * a) % p) & scale
    return my_hash

In [84]:
def distinct_vals(seq, a):
    if len(seq) < 15: return 0
    i = 0
    hashes = set()
    h_num = get_ath_hash(a)
    while i + 15 <= len(seq):
        subseq = seq[i:i+15]
        i += 1
        # print(subseq, i)
        hashes.add(h_num(subseq))
    return len(hashes)

# test on simple data
data_1 = "abcdefghijklmnopqabcdefghijklmno"
data_2 = "ctagctagctagctagctagcccccccccccc"
data_3 = "acacacacacacacacacacacacacacacac"
print("test 1: Correct answer should be 17")
print("Observed answer is", distinct_vals(data_1, 10), "\n")
print("test 2: Correct answer should be 15")
print("Observed answer is", distinct_vals(data_2, 10), "\n")
print("test 3: Correct answer should be 2")
print("Observed answer is", distinct_vals(data_3, 10), "\n")

# test on chromosome
for chromosome in human_genome:
    if chromosome.name == "CM000664.2":
        sequence = str(chromosome.seq).lower()
        # note that 1 billion is approxiamtely 2^30
        print("test 4: trying algorithm on chromosome 2")
        print(distinct_vals(sequence, 30))

test 1: Correct answer should be 17
Observed answer is 17 

test 2: Correct answer should be 15
Observed answer is 15 

test 3: Correct answer should be 2
Observed answer is 2 

test 3: trying algorithm on chromosome 2
144698064
